# Context

Here we use the expected images (polynomial fitted) to compute the contrast.

# Imports

In [ ]:
from skimage import io
import skimage
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, uniform_filter
import pickle

In [ ]:
import imageio
from pathlib import Path
from matplotlib.pyplot import show
from argparse import ArgumentParser

from pyoptflow.plots import compareGraphs

In [ ]:
from PIL import Image
import os
from scipy.signal import argrelextrema
from skimage import exposure

In [ ]:
import matplotlib
import matplotlib.animation
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128

### Import our custom utility methods

In [ ]:
import sys
%reload_ext autoreload
%autoreload 2
sys.path.append('..')

from utils.visualization_tools import *
import utils.visualization_tools
from utils.data_transformations import *
import utils.data_transformations
from utils.diverse import *
import utils.diverse

The following modules are available

In [ ]:
print_module_methods(utils.diverse)

In [ ]:
print_module_methods(utils.visualization_tools)

In [ ]:
print_module_methods(utils.data_transformations)

In [ ]:
from pathlib import Path
source_folder = os.path.join(Path(os.getcwd()).parent, "datasets/source_data")

files = []
files.append(os.path.join(source_folder,"runstart16_X1.tif"))
files.append(os.path.join(source_folder,"runstart16_X2.tif"))

filepath = files[0]

In [ ]:
frames = np.array(skimage.io.imread(os.path.join(filepath)), dtype=np.double)[:1000]
mean = np.mean(frames,axis=0)#pixelwise mean
frames = framewise_difference(frames, mean, bigdata=True)
mean = None

In [ ]:
if False:#This leads to an artifact probably due to gibbs phenomenom
    frames = remove_frequency_from_pixel_vectors(frames,15,20)

In [ ]:
#frames = gaussian_filter(frames,2)

# It is aimed for removing the expected image from each frame

Here is an example.

In [ ]:
#expected = np.load("expected_images.npy")
expected = np.load("10_bins_expected_images_polynomial.npy")

In [ ]:
#np.argmin(np.abs(expected_mean-np.mean(frames[0])))#45

In [ ]:
plt.imshow(frames[0])

In [ ]:
plt.imshow(expected[0])

In [ ]:
np.mean(frames,axis=(1,2))

In [ ]:
np.mean(expected,axis=(1,2))

Now we do so for all frames.

In [ ]:
#frames = raw_frames.copy()

In [ ]:
raw_frames = frames.copy()

In [ ]:
frames = gaussian_filter(frames,2)

In [ ]:
#expected = np.load("expected_images.npy")

In [ ]:
expected = gaussian_filter(expected,2)

In [ ]:
if False:
    mean = np.mean(frames,axis=(1,2))#pixelwise mean
else:
    mean = np.median(frames,axis=(1,2))

In [ ]:
plt.plot(mean)

In [ ]:
if False:
    expected_mean = np.mean(expected, axis=(1,2))
else:
    expected_mean = np.median(expected, axis=(1,2))

In [ ]:
plt.plot(expected_mean)

In [ ]:
#compute mapping: for each mean it specifies the corresponding index of the expected images 
column_vectors = np.tile(np.array([expected_mean]).transpose(), (1, len(mean)))
mapping = np.argmin(np.abs(column_vectors - mean), axis=0)

In [ ]:
for i, expected_image_idx in enumerate(mapping):
    frames[i] -= expected[expected_image_idx]

In [ ]:
[np.mean(f) for f in expected][46]

In [ ]:
np.min(frames)

In [ ]:
#frames -= np.mean(frames, axis=0)

In [ ]:
#frames = remove_frequency_from_pixel_vectors(frames,15,20)

In [ ]:
%%capture
ani = show_video(normalize(frames)[550:600+100], n_frames=100, vmin = .1, vmax=1.0)

In [ ]:
HTML(ani)

In [ ]:
%%capture
ani = show_video(normalize(frames)[560:560+150], n_frames=150, vmin = 0, vmax=1.0)

In [ ]:
HTML(ani)

In [ ]:
import numpy as np

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def smooth_binarization(frames, k = 10):
    frames = (normalize(frames) * k)-(k/2)
    frames = np.array([sigmoid(f) for f in frames])
    return frames

In [ ]:
roi = smooth_binarization(frames)

In [ ]:
np.save("sigmoid_test.npy",roi)

In [ ]:
plt.plot(np.linspace(-10,10,100),sigmoid(np.linspace(-10,10,100)))

In [ ]:
#plt.plot(normalize(-np.mean(frames, axis=(1,2))))
plt.plot(normalize(np.mean(raw_frames, axis=(1,2))))
plt.plot(normalize(np.mean(roi, axis=(1,2))))

Mind that the frame means of the contrast to the expected image are negative (Larger regions with less activation as compared to the expectation and small focal peaks. Due to sigmoid transform these large regions have a negative value. Hence after normalization there is a negative peak for up-state periods). During down states the difference is closer to zero. Hence after normalization up states are characterized by frame means of one.

In [ ]:
np.median(raw_frames[0,:50,:50]-expected[10,:50,:50])

In [ ]:
%%capture
ani = show_video(roi, n_frames=800, vmin=0, vmax=1)

In [ ]:
HTML(ani)

In [ ]:
# use median!